## Flask-WebAuthn
extension that enables WebAuthn support for passwordless authentication in Flask applications. WebAuthn is a standard that allows users to authenticate using biometrics (like fingerprints) or hardware security keys (like YubiKey) instead of traditional passwords.

#### WebAuthn Workflow:
- **Credential Registration**: The user registers a new credential (e.g., a fingerprint or security key) that will be used for authentication.
- **Authentication**: When logging in, the user proves they own the credential (e.g., by pressing a button on their hardware key or using their fingerprint).

In [ ]:
# Installation 
!pip install Flask-WebAuthn

In [ ]:
from flask import Flask, jsonify, request
from flask_webauthn import WebAuthn

app = Flask(__name__)
app.secret_key = 'another_random_secret_key'

# Initialize WebAuthn with the Flask app
web_authn = WebAuthn(app)

### Define WebAuthn Options
The following options specify the details for credential creation, like the relying party (RP) ID, which typically matches your domain.

In [ ]:
web_authn_options = {
    "rp_id": "localhost",  # The domain or host serving the app
    "rp_name": "WebAuthn Example",  # The name of your app
    "user_id": "user@example.com",  # Unique user ID in your system
    "user_name": "User",  # The display name for the user
    "challenge": "some_random_challenge_string"  # A challenge string to prevent replay attacks
}

### Setting Template

WebAuthn registration is triggered after the form is submitted. The WebAuthn API is then used to generate options for creating a WebAuthn credential.

### Handling Registration

- **Handling Registration**
This route prepares the WebAuthn creation options and sends them to the client.

In [ ]:
@app.route('/register', methods=['POST'])
def register():
    # Extract user details (user_id and username) from the request data.
    user_id = request.json.get('user_id')
    username = request.json.get('username')

    # Update WebAuthn options with the user details.
    web_authn_options["user_id"] = user_id
    web_authn_options["user_name"] = username

    # Prepare the options for creating a new WebAuthn credential
    options = web_authn.prepare_create(user_id=user_id, username=username)
    
    # Send these options to the frontend to initiate the registration process
    return jsonify(options)

- **Verify Credential Creation**
After the user registers a new credential (e.g., by using their fingerprint or security key), the client sends the credential data back to the server. The server verifies it.

In [ ]:
@app.route('/register/verify', methods=['POST'])
def verify_register():
    # Receive the user's credential creation data from the frontend
    credential = request.json.get('credential')

    # Verify the credential (checks the public key, signature, etc.)
    success = web_authn.verify_create(credential)
    if success:
        return 'Credential successfully created!'  # Confirmation message
    else:
        return 'Error creating credential.'


### Handling Authentication

- **Create Authentication Route**: This route prepares the challenge to authenticate the user. It sends a challenge to the frontend, where the user will authenticate using their previously registered credential.

In [ ]:
@app.route('/login', methods=['POST'])
def login():
    user_id = request.json.get('user_id')
    
    # Prepare the authentication challenge for the user
    options = web_authn.prepare_assert(user_id=user_id)
    
    return jsonify(options)  # Send challenge to frontend to start authentication

- **Verify Authentication**: After the user responds to the challenge (by pressing their hardware key or using biometrics), the credential is sent back to the server for verification.

In [ ]:
@app.route('/login/verify', methods=['POST'])
def verify_login():
    # Receive the authentication data from the frontend
    credential = request.json.get('credential')

    # Verify the assertion (check the user's response to the authentication challenge)
    success = web_authn.verify_assert(credential)
    if success:
        return 'Authentication successful!'  # Success message
    else:
        return 'Authentication failed.'  # Failure message